In [1]:
import requests
import pandas as pd
import geopandas as gpd
from datetime import datetime, timedelta

In [2]:
main = gpd.read_file('./full_globfire/full_globfire.shp')

# Drop bottom 20% area fires
bottom = main['area_ha'].quantile(0.2)
main = main[main['area_ha'] > bottom]

# Add bounding boxes for scrape
main = pd.concat([main, main.bounds], axis=1)

In [3]:
main.head()

,_uid_,id,initialdat,finaldate,area_ha,geometry,minx,miny,maxx,maxy
1,17.0,966856.0,2002-01-10,2002-01-10,279.078281,"MULTIPOLYGON (((-119.73174 35.63750, -119.7214...",-119.731744,35.616667,-119.696533,35.641667
2,18.0,966858.0,2002-01-11,2002-01-14,364.862618,"MULTIPOLYGON (((-119.48016 34.59167, -119.4852...",-119.504140,34.591667,-119.459916,34.608333
3,19.0,966859.0,2002-01-08,2002-01-15,171.639568,"POLYGON ((-113.32725 32.97917, -113.32228 32.9...",-113.327632,32.975000,-113.302037,32.983333
4,45798.0,966857.0,2002-01-17,2002-01-17,193.206072,"MULTIPOLYGON (((-119.72149 35.63750, -119.7112...",-119.730628,35.616667,-119.704996,35.641667
5,78962.0,1053315.0,2002-02-06,2002-02-10,214.857839,"MULTIPOLYGON (((-121.77411 39.42083, -121.7633...",-121.774111,39.416667,-121.736355,39.433333


## Vertical Profiles Available


In [4]:
profiles_of_interest = [
    'air',  # Air Temperature
    'hgt',  # Geo-potential Height
    'omega',  # Vertical Velocity in Pressure Coordinates
    'shum',  # Specific Humidity
    'tke',  # Turbulent Kinetic Energy
    'uwnd',  # U-component of wind
    'vwnd'  # V-component of wind
]

In [5]:
import os

for profile in profiles_of_interest:
    os.makedirs(f'./assets/{profile}', exist_ok=True)

In [6]:
URL = "https://psl.noaa.gov/thredds/ncss/grid/Datasets/NARR/pressure/{profile}.{yearmonth}.nc?north={N}&west={W}&east={E}&south={S}&horizStride=1&time_start={start}Z&time_end={end}Z&&&accept=netcdf4-classic"